<h1>Kamus Bahasa Jawa</h1>
<p>Dibuat oleh: <a href="https://linkedin.com/n/adityahanif/">Aditya Hanif</a>.

Pada kali ini saya akan membuat kamus bahasa kawi (bahasa jawa kuno) dengan bahan yang berasal dari website <a href="https://sastra.org/">sastra.org</a>. Yang saya lakukan adalah dengan scrapping data yang terdapat pada website tersebut.

Berikut adalah langkah-langkahnya.</p>

<h2>1. Import Library</h2>
<p>Library yang saya gunakan adalah Request, Re dan BeautifulSoup</p>

In [25]:
import requests
from bs4 import BeautifulSoup
import re

<h2>2. Listing URL</h2>

<p>Mengumpulkan URL mana saja yang akan dijadikan sumber entry</p>

In [26]:
list_url = ["https://www.sastra.org/bahasa-dan-budaya/kamus-dan-leksikon/2692-kawi-indonesia-wojowasito-1977-1019-a-c.html",
       "https://www.sastra.org/bahasa-dan-budaya/kamus-dan-leksikon/2695-kawi-indonesia-wojowasito-1977-1019-d-k.html",
       "https://www.sastra.org/bahasa-dan-budaya/kamus-dan-leksikon/2697-kawi-indonesia-wojowasito-1977-1019-l-p.html",
       "https://www.sastra.org/bahasa-dan-budaya/kamus-dan-leksikon/2699-kawi-indonesia-wojowasito-1977-1019-r-t.html",
       "https://www.sastra.org/bahasa-dan-budaya/kamus-dan-leksikon/3074-kawi-indonesia-wojowasito-1977-1019-u-y.html"]

<h2>3. Membuat Fungsi Parsing</h2>
<p>Agar dapat melakukan parsing tag div dan dl sekaligus pada semua url yang terdapat di list url</p>

In [27]:
def parse(x):
  result = ""
  for index, url in enumerate(list_url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    div = soup.find("div", id="mark_range")
    paragraphs = div.find_all('dl')
    for para in paragraphs:
      entry_text = para
      result += str(entry_text)
  return result


<h2>4. Parsing HTML</h2>
<p>Melakukan parsing html di semua URL menggunakan fungsi yang telah dibuat</p>

In [28]:
kawi = parse(list_url)
print(kawi[:500])

<dl class="ysl-dl-hrz">
<dt>a</dt><dd>I. awalan katakerja aktif; misalnya: <i>gawe</i> → <i>agawe</i> berbuat, membuat; II. akhiran irrealis dengan arti: <em>agar supaya</em>, <em>hendak</em>, <em>dianggap sebagai</em>, <em>berharap</em>, misalnya: alun → <i>aluna</i> seperti ombak; III. awalan dengan arti: <em>tidak</em>, <em>tanpa</em>, misalnya: <i>wighnam</i> rintangan; <i>awighnam</i> tanpa rintangan.</dd>
<dt>abab</dt><dd><i>mābab-abab</i> dalam aturan; teratur.</dd>
<dt>abah</dt><dd><i>ab


<h2>5. Membuat Fungsi Untuk Cleaning</h2>
<p>Seluruh hasil yang telah dilakukan parsing belum sepenuhnya bersih serta masih menyisakan tag dl dan dt. Sebenarya tag tersebut bisa langsung dihilangkan di awal (saat membuat fungsi parsing) dengan cara menambahkan method .text.strip().
Contoh seperti ini:

&lt;dt&gt;abda&lt;/dt&gt;&lt;dd&gt;S. awan; musim hujan.&lt;/dd&gt;
<br />
&lt;dt&gt;diwangkara&lt;/dt&gt;&lt;dd&gt;matahari.&lt;/dd&gt;

Bila tag diatas langsung dihilangkan dengan .text.strip() maka hasilnya adalah demikian

abda S. awan; musim hujan.<br />
diwangkara matahari.

Sehingga akan lebih sulit untuk memberi tanda ':' sebagai pemisah kata dan artinya.

Maka dari itu akan lebih mudah mengubah tag &lt;/dd&gt;&lt;dt&gt; menjadi tanda ':'

Selain contoh di atas masih ada beberapa sisa tag HTML lain yang harus dibersihkan</p>

In [29]:
def clean(x):
  x = re.sub("</dt><dd>", ": ", x)
  x = re.sub("<dt>", "", x)
  x = re.sub("</dd>", "", x)
  x = re.sub("<dd>", "", x)
  x = re.sub("<i>", "(", x)
  x = re.sub("</i>", ")", x)
  x = re.sub("S.", "", x)
  x = re.sub("  ", " ", x)
  x = re.sub("\<.*?\>","<>",x)
  x = re.sub("\>.*?\<","><",x)
  x = re.sub("\n\n", "", x)
  x = re.sub("<>: ", "", x)
  x = re.sub("<>", "", x)
  x = re.sub("\[.*?\]","[]", x)
  x = re.sub("\n\n", " ", x)
  x = re.sub("\n ", "", x)
  x = re.sub("\r\n\r", "", x)
  x = re.sub("→ ", "", x)
  return x

<h2>6. Membuat Variabel Berisi Data Bersih</h2>
<p>Setelah fungsi dibuat maka selanjutnya adalah membuat variabel baru bernama kamus, isinya adalah data yang telah dibersihkan dengan fungsi clean()</p>

In [30]:
kamus = clean(kawi)
kamus[:500]

'\na: I. awalan katakerja aktif; misalnya: (gawe) (agawe) berbuat, membuat; II. akhiran irrealis dengan arti: , misalnya: (wighnam) rintangan; (awighnam) tanpa rintangan.\nabab: (mābab-abab) dalam aturan; teratur.\nabah: (abah-abah) pakaian kuda; abah-abah.\nabala: lemah; lembek.\nabalang: lempar; buang; (inabalangakěn) dilempar; dibuang.\nābandha: Ikatan.\nabar: (angabar-abar) bersinar-sinar.\nabda: awan; musim hujan.\nabdhi: lautan.\nabdhikanyā: dewi Laksmi.\naběn: (mengaběn) melempari.\naběr: (angaběrakén'

<h2>7. Menyimpan Hasil</h2>
<p>Hasil akan diekspor ke dalam bentuk file, dalam hal ini bisa berupa .txt maupun .html.

Sebelum disimpan alangkah baiknya dibedakan format penulisan antara entri kata dan artinya. Untuk entri kata akan dicetak bold.

Karenanya perlu dibuat fungsi terlebih dulu supaya setiap entri kata akan diapit tag &lt;b&gt;&lt;/b&gt; dan disimpan sebagai .html sehingga ketika dicopy ke Microsoft Word semua entry sudah berformat bold</p>

In [31]:
# Fungsi untuk membuat teks tebal
def bold_match(matchobj):
    return "<b>{}:</b>".format(matchobj.group(1))

# Menggunakan regular expression untuk mencari pola teks sebelum tanda ":"
final_result = re.sub(r"^(.*?):", bold_match, kamus, flags=re.MULTILINE)

final_result = re.sub("<b>", "<br /><b>", final_result)

def capitalize(entrytext):
    return entrytext.group(0).upper()
def lower(entrytext):
    return entrytext.group(0).lower()

final_result = re.sub(r">.", capitalize, final_result)
final_result = re.sub(r"</b> .", lower, final_result)
final_result = re.sub("</b> i. ", "</b> I. ", final_result)

# Output
with open('Kamus Kawi Indonesia Capitalized.html', 'w') as file:
    file.write(final_result)

<h2>Final</h2>

<p>Selanjutnya bisa diedit di Microsoft Word dengan beberapa penyesuaian seperti menambahkan pendahuluan, memasukkan indeks, dan lainnya.

Adapun untuk filenya bisa didownload di sini:<br />
<a href="https://github.com/adityahanif/portfolio/blob/main/file/Kamus%20Kawi%20Indonesia%20Capitalized.html/">Kamus Kawi Indonesia HTML</a> (File Hasil Scrap Dari Proses Ini)<br />
<a href="https://github.com/adityahanif/portfolio/blob/main/file/Kamus%20Kawi%20Indonesia%20(capitalized).pdf/">Kamus Kawi Indonesia PDF</a> (File Akhir Yang Telah Diedit dan Berbentuk PDF)<br />
<br />
Kami ucapkan terima kasih banyak kepada <a href="https://sastra.org/">sastra.org</a><br />
<br />
P.S. Hasil tidak dikomersilkan dan tidak untuk dikomersilkan oleh siapapun. Hanya untuk tujuan edukasi.
</p>